In [ ]:
pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 50.8 MB/s eta 0:00:00


In [ ]:
import pulp
import ast
import numpy as np
import pandas as pd
import random
import os
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_datetime64_any_dtype
from datetime import datetime, timedelta, date
import time
import math
demand = [1,3,4,6,3,7,8,4,2,4,3]
valid_shifts = [(1,3),(2,5),(4,5),(5,11),(7,9),(3,11)]
valid_supply = {}
for i in range(1,12):
    shift_list = []
    for shift in valid_shifts:
        if (shift[0]<=i) and (shift[1]>=i):
            shift_list.append(shift)
    valid_supply[i] = shift_list
prob = pulp.LpProblem("Minimize_supply_for_demand", pulp.LpMinimize)

x = pulp.LpVariable.dicts("supply",[
                       shift for shift in valid_shifts
                     ],lowBound = 0,cat = "Continuous")

# objective function
prob += pulp.lpSum([x[shift] for shift in valid_shifts])

# constraint to meet demand
for i in range(1,12):
    prob += pulp.lpSum([x[shift] for shift in valid_supply[i]]) >= demand[i-1]
prob.solve()

1

In [ ]:
for shift in valid_shifts:
  print(pulp.value(x[shift]))

1.0
2.0
0.0
3.0
1.0
4.0


In [ ]:
# creating a dummy dataframe with time and number of people required at each hour
interaction_volume = pd.DataFrame()
# Total number of half hours in a week is 336
interaction_volume['time'] = np.arange(1,337,dtype = int)
# Add the corresponding day to each half hour time step
interaction_volume['day'] = interaction_volume['time'].apply(lambda x: math.ceil(x/49))
# Predicted number of users required at each half hour
interaction_volume['number_of_users'] = np.random.uniform(10,20,len(interaction_volume)).round(0)

In [ ]:
interaction_volume

,time,day,number_of_users
0,1,1,19.0
1,2,1,13.0
2,3,1,12.0
3,4,1,15.0
4,5,1,11.0
...,...,...,...
331,332,7,14.0
332,333,7,13.0
333,334,7,13.0
334,335,7,13.0


In [ ]:
# function to read dataset from Ikigai project
def get_dataset(dataset_id):
    '''
    Read data from Ikigai datasets
    Make sure the aisuccess account has the access to the project/dataset
    Input the dataset_id as string from the url for ex: *******
    '''
    import requests
    import pandas as pd
    url = "" + dataset_id
    headers = {
            'User': '*****',
            'api-key': '******'
            }

    response = requests.request("GET", url, headers=headers)
    #print(response)
    data = pd.read_csv(response.json()['url'], low_memory = False)
    return data

In [ ]:
data = get_dataset('2Z83BhyqKG3uCA5mkbm9lzsDMDz')

In [ ]:
# data = pd.read_csv('/content/sample_data/user_requirements_dec1.csv',low_memory = False)

In [ ]:
data = data.iloc[:336]

In [ ]:
data

,Date,Interaction volume,raw_positions,positions,service_level,occupancy,waiting_probability
0,2023-10-16 00:00:00,21.990606,8.0,10.0,0.834430,0.641393,0.187232
1,2023-10-16 00:30:00,22.373208,8.0,10.0,0.820997,0.652552,0.201649
2,2023-10-16 01:00:00,23.970543,9.0,12.0,0.878359,0.621459,0.140763
3,2023-10-16 01:30:00,25.918447,9.0,12.0,0.820919,0.671960,0.203236
4,2023-10-16 02:00:00,27.211513,10.0,13.0,0.883398,0.634935,0.136349
...,...,...,...,...,...,...,...
331,2023-10-22 21:30:00,52.539102,16.0,20.0,0.800545,0.766195,0.234139
332,2023-10-22 22:00:00,39.914328,13.0,17.0,0.835926,0.716411,0.192158
333,2023-10-22 22:30:00,27.497176,10.0,13.0,0.876623,0.641601,0.143860
334,2023-10-22 23:00:00,22.984403,9.0,12.0,0.901879,0.595892,0.114672


In [ ]:
data = data.reset_index().rename(columns = {'index': 'time'})

In [ ]:
data['time'] = data['time'] + 1

In [ ]:
data['day'] = data['time'].apply(lambda x: math.ceil(x/49))

In [ ]:
interaction_volume = data[['day','time','raw_positions']]

In [ ]:
interaction_volume = interaction_volume.rename(columns = {'raw_positions':'number_of_users'})

In [ ]:
# Create valid time_shifts
valid_time_shifts = set ()
end_time_duration_list_part_time = [20,12,14] # different possible end time durations for part time employees
end_time_duration_list_full_time = [17,16] # different possible end time durations for full time employees
end_day_duration_list_part_time = [3,4] # different possible end day durations for part time employees
end_day_duration_list_full_time = [4] # different possible end day durations for full time employees
valid_end_times_pt_4 = [20] # valid end_times when total number of working days is 4 for part time employees
valid_end_times_pt_5 = [12,14] # valid end_times when total number of working days is 5 for part time employees
valid_end_times_ft_4 =[] # valid end_times when total number of working days is 4 for full time employees
valid_end_times_ft_5 = [17,16] # valid end_times when total number of working days is 5 for part time employees
worker_type = [0,1] # 0 means full_time 1 means part_time

In [ ]:
# creating valid shifts using above data
for worker in worker_type:
    if worker == 0:
        end_day_duration_list = end_day_duration_list_full_time
    elif worker == 1:
        end_day_duration_list = end_day_duration_list_part_time
    for start_day in range(1,8):
        for end_day_duration in end_day_duration_list:
            if (worker == 0) and (end_day_duration ==4):
                end_time_duration_list = valid_end_times_ft_5
            if (worker == 1) and (end_day_duration == 3):
                end_time_duration_list = valid_end_times_pt_4
            if (worker == 1) and (end_day_duration == 4):
                end_time_duration_list = valid_end_times_pt_5
            for start_time in range(1,49):
                for end_time_duration in end_time_duration_list:
                    valid_time_shifts.add((worker,start_day,start_day+end_day_duration,start_time,start_time+end_time_duration))

In [ ]:
# len(valid_time_shifts)

In [ ]:
(1, 1, 5, 29, 43) in valid_time_shifts

True

In [ ]:
# valid_time_shifts

In [ ]:
%%time
# Find valid variables for each shift
# Possible people who can work in each time slot
valid_variables_time_shift = {}
for i in range(336) :
    current_time = interaction_volume.iloc[i]['time']
    current_day = interaction_volume.iloc[i]['day']
    shift_list = []

    for worker in worker_type:
        # Each type of user has a different work duration list
        if worker == 0:
            end_day_duration_list = end_day_duration_list_full_time
        elif worker == 1:
            end_day_duration_list = end_day_duration_list_part_time

        for start_day in range(1,8):

            for end_day_duration in end_day_duration_list:

                if (worker == 0) and (end_day_duration ==4):
                    end_time_duration_list = valid_end_times_ft_5
                if (worker == 1) and (end_day_duration == 3):
                    end_time_duration_list = valid_end_times_pt_4
                if (worker == 1) and (end_day_duration == 4):
                    end_time_duration_list = valid_end_times_pt_5

                for start_time in range(1,49):

                    for end_time_duration in end_time_duration_list:
                        # We are using duration to find the end_times
                        end_day = start_day+end_day_duration
                        end_time = start_time+end_time_duration

                        if (start_day <= current_day) and (end_day >= current_day):
                            # converting time of each day to time of week
                            a = start_time + (current_day-1)*48
                            b = end_time + (current_day-1)*48

                            if ((a <= current_time) and (b >= current_time)):
                                shift_list.append((worker,start_day,end_day,start_time,end_time))

                            if (start_day <= (current_day-1)) and (((b-48) >=current_time)):
                              shift_list.append((worker,start_day,end_day,start_time,end_time))

                        if (start_day >= current_day) and ((end_day-7) >= current_day):
                            # making the shifts periodic so that people as people who start working at end of week is working
                            # is also working first few days of the week
                            # if end day is 9 means that person is working 2 days after last day of week

                            start_day_periodic = start_day-7
                            end_day_periodic = end_day-7
                            a = start_time + (current_day-1)*48
                            b = end_time + (current_day-1)*48

                            if ((a <= current_time) and (b >= current_time)):
                              shift_list.append((worker,start_day,end_day,start_time,end_time))

                            if (start_day_periodic <= (current_day-1)) and (((b-48) >=current_time)):
                              shift_list.append((worker,start_day,end_day,start_time,end_time))



    valid_variables_time_shift[current_time] = shift_list

CPU times: user 893 ms, sys: 13 ms, total: 906 ms
Wall time: 908 ms


In [ ]:
# (0,7,11,39,53) in valid_time_shifts

In [ ]:
# valid_variables_time_shift[1]

In [ ]:
def time_shift(shift):
  x = (shift[2] - shift[1])*(shift[4]- shift[3])
  return x

In [ ]:
# time_shift(shift)

In [ ]:
# valid_time_shifts

In [ ]:
# vaild_shift_new = set()

# for key, shift_list in valid_variables_time_shift.items():
#     for shift in shift_list:
#         vaild_shift_new.add(shift)

In [ ]:
# valid_shift_new = set()
# for key,shift_list in valid_variables_time_shift.items():
#   for shift in shift_list:
#     vaild_shift_new.add(shift)

In [ ]:
# vaild_shift_new

In [ ]:
%%time

####################
#   PROBLEM SETUP  #
####################

# Create the 'prob' variable to contain the problem data
prob = pulp.LpProblem("Minimise_total_number_of_agents", pulp.LpMinimize)

# define the lp variables
x = pulp.LpVariable.dicts("shift", [
            shift for shift in valid_time_shifts
            ], lowBound=0 ,cat='Integer')

# objective function
# Minimise total number of hour worked to meet the forecasted demand
prob += pulp.lpSum([time_shift(shift)*x[shift] for shift in valid_time_shifts ])

#
for key,shift_list in valid_variables_time_shift.items():
    prob += pulp.lpSum([x[shift] for shift in shift_list if shift in valid_time_shifts])>= (interaction_volume.iloc[int(key)-1]['number_of_users'])

# part_time workers worker_type = 1 can't be more than 50 % of the workforce
prob += pulp.lpSum([-1*x[shift] for shift in valid_time_shifts if shift[0] == 1 ]) + pulp.lpSum([x[shift] for shift in valid_time_shifts if shift[0] == 0 ]) >= 0


solver = pulp.PULP_CBC_CMD(msg = True,timeLimit=2000, threads=8, gapRel=.0005)
prob.solve(solver)

CPU times: user 1.29 s, sys: 86.3 ms, total: 1.37 s
Wall time: 1min 44s


1

In [ ]:
shift[0]

1

In [ ]:
# %%time

# ####################
# #   PROBLEM SETUP  #
# ####################

# # Create the 'prob' variable to contain the problem data
# prob = pulp.LpProblem("Minimise_total_number_of_agents", pulp.LpMinimize)

# # define the lp variables
# x = pulp.LpVariable.dicts("shift", [
#             shift for shift in vaild_shift_new
#             ], lowBound=0 ,cat='Integer')

# # objective function
# prob += pulp.lpSum([time_shift(shift)*x[shift] for shift in vaild_shift_new ])

# #
# for key,shift_list in valid_variables_time_shift.items():
#     prob += pulp.lpSum([x[shift] for shift in shift_list if shift in vaild_shift_new])>= (interaction_volume.iloc[int(key)-1]['number_of_users'])

# solver = pulp.PULP_CBC_CMD(msg = True,timeLimit=2000, threads=8, gapRel=.1)
# prob.solve(solver)

In [ ]:
[shift for shift in shift_list if shift in valid_time_shifts]

[(0, 3, 7, 31, 48),
 (0, 3, 7, 32, 49),
 (0, 3, 7, 32, 48),
 (0, 3, 7, 33, 50),
 (0, 3, 7, 33, 49),
 (0, 3, 7, 34, 51),
 (0, 3, 7, 34, 50),
 (0, 3, 7, 35, 52),
 (0, 3, 7, 35, 51),
 (0, 3, 7, 36, 53),
 (0, 3, 7, 36, 52),
 (0, 3, 7, 37, 54),
 (0, 3, 7, 37, 53),
 (0, 3, 7, 38, 55),
 (0, 3, 7, 38, 54),
 (0, 3, 7, 39, 56),
 (0, 3, 7, 39, 55),
 (0, 3, 7, 40, 57),
 (0, 3, 7, 40, 56),
 (0, 3, 7, 41, 58),
 (0, 3, 7, 41, 57),
 (0, 3, 7, 42, 59),
 (0, 3, 7, 42, 58),
 (0, 3, 7, 43, 60),
 (0, 3, 7, 43, 59),
 (0, 3, 7, 44, 61),
 (0, 3, 7, 44, 60),
 (0, 3, 7, 45, 62),
 (0, 3, 7, 45, 61),
 (0, 3, 7, 46, 63),
 (0, 3, 7, 46, 62),
 (0, 3, 7, 47, 64),
 (0, 3, 7, 47, 63),
 (0, 3, 7, 48, 65),
 (0, 3, 7, 48, 64),
 (0, 4, 8, 31, 48),
 (0, 4, 8, 32, 49),
 (0, 4, 8, 32, 48),
 (0, 4, 8, 33, 50),
 (0, 4, 8, 33, 49),
 (0, 4, 8, 34, 51),
 (0, 4, 8, 34, 50),
 (0, 4, 8, 35, 52),
 (0, 4, 8, 35, 51),
 (0, 4, 8, 36, 53),
 (0, 4, 8, 36, 52),
 (0, 4, 8, 37, 54),
 (0, 4, 8, 37, 53),
 (0, 4, 8, 38, 55),
 (0, 4, 8, 38, 54),


In [ ]:
# for key,shift_list in valid_variables_time_shift.items():
#   for shift in shift_list:
#     if (shift[4] -shift[3]) != 20:
#       print(shift)

In [ ]:
# (shift[2] - shift[1])*(shift[4]- shift[3])

In [ ]:
# prob

In [ ]:
# pulp.value(x[(0,2,6,10,27)])

In [ ]:
optimised_shift = {}
all_optimised_shifts = []
worker_type = []
start_day = []
end_day = []
start_time = []
end_time = []
for shift in valid_time_shifts:
  if pulp.value(x[shift])>0.0:
    print(shift,pulp.value(x[shift]))
    # a = str(shift)
    optimised_shift = {'shift': shift , 'number_of_users': pulp.value(x[shift])}
    all_optimised_shifts.append(optimised_shift)

(1, 1, 4, 16, 36) 5.0
(1, 1, 4, 22, 42) 1.0
(1, 6, 9, 20, 40) 1.0
(1, 5, 8, 17, 37) 2.0
(0, 4, 8, 18, 35) 1.0
(0, 6, 10, 13, 29) 1.0
(0, 7, 11, 35, 52) 1.0
(1, 5, 8, 19, 39) 4.0
(0, 6, 10, 15, 31) 1.0
(0, 1, 5, 27, 43) 3.0
(1, 1, 4, 18, 38) 4.0
(1, 1, 4, 19, 39) 4.0
(1, 1, 5, 14, 26) 4.0
(0, 1, 5, 23, 40) 2.0
(0, 4, 8, 7, 24) 1.0
(1, 6, 9, 16, 36) 2.0
(0, 6, 10, 17, 34) 1.0
(0, 1, 5, 29, 45) 6.0
(1, 1, 4, 20, 40) 2.0
(0, 5, 9, 14, 30) 3.0
(1, 6, 9, 18, 38) 2.0
(0, 1, 5, 32, 48) 2.0
(1, 5, 8, 15, 35) 1.0
(0, 5, 9, 16, 33) 3.0
(0, 6, 10, 22, 39) 1.0
(0, 7, 11, 13, 29) 4.0
(0, 1, 5, 21, 38) 2.0
(0, 5, 9, 10, 26) 1.0
(1, 7, 10, 20, 40) 1.0
(0, 1, 5, 36, 53) 1.0
(1, 6, 9, 14, 34) 1.0
(1, 2, 6, 14, 26) 3.0
(1, 6, 9, 36, 56) 1.0
(0, 1, 5, 34, 50) 1.0
(1, 5, 9, 13, 25) 1.0
(0, 6, 10, 18, 35) 1.0
(1, 1, 4, 32, 52) 1.0
(0, 6, 10, 24, 41) 2.0
(1, 5, 8, 20, 40) 3.0
(0, 6, 10, 32, 48) 1.0
(0, 7, 11, 31, 47) 3.0
(0, 1, 5, 5, 22) 2.0
(1, 1, 5, 30, 42) 3.0
(0, 1, 5, 12, 28) 3.0
(0, 5, 9, 23, 40) 1.0
(

In [ ]:
# optimised_shift = {}
# all_optimised_shifts = []
# worker_type = []
# start_day = []
# end_day = []
# start_time = []
# end_time = []
# for shift in vaild_shift_new:
#   if pulp.value(x[shift])>0.0:
#     print(shift,pulp.value(x[shift]))
#     # a = str(shift)
#     optimised_shift = {'shift': shift , 'number_of_users': pulp.value(x[shift])}
#     all_optimised_shifts.append(optimised_shift)

In [ ]:
supply = {}
for i in range(1,337):
  valid_shifts = valid_variables_time_shift[i]
  count = 0
  for shift in valid_time_shifts:
    if pulp.value(x[shift])>0.0:
      if shift in valid_shifts:
        count += pulp.value(x[shift])
  supply[i] = count

In [ ]:
# valid_shifts

In [ ]:
optimised_df = pd.DataFrame(all_optimised_shifts)

In [ ]:
# optimised_df.to_csv('optimised_df.csv',index = False)

In [ ]:
# supply = {}
# for i in range(1,337):
#   valid_shifts = valid_variables_time_shift[i]
#   count = 0
#   for shift in valid_shift_new:
#     if pulp.value(x[shift])>0.0:
#       if shift in valid_shift_new:
#         count += pulp.value(x[shift])
#   supply[i] = count

In [ ]:
pd.DataFrame(supply,index = [0]).T.reset_index(drop = True).rename(columns = {0: 'scheduled'})

,scheduled
0,8.0
1,8.0
2,9.0
3,9.0
4,10.0
...,...
331,22.0
332,19.0
333,19.0
334,12.0


In [ ]:
final = pd.concat([interaction_volume,pd.DataFrame(supply,index = [0]).T.reset_index(drop = True).rename(columns = {0: 'scheduled'})],axis = 1)

In [ ]:
data['Date'].min()

'2023-10-16 00:00:00'

In [ ]:
final['Date'] = final['time'].apply(lambda x: pd.to_datetime(data['Date'].min())+ timedelta(hours = (x-1)*0.5))

In [ ]:
final.to_csv('schedule_test.csv',index = False)

In [ ]:
final

,day,time,number_of_users,scheduled,Date
0,1,1,8.0,8.0,2023-10-16 00:00:00
1,1,2,8.0,8.0,2023-10-16 00:30:00
2,1,3,9.0,9.0,2023-10-16 01:00:00
3,1,4,9.0,9.0,2023-10-16 01:30:00
4,1,5,10.0,10.0,2023-10-16 02:00:00
...,...,...,...,...,...
331,7,332,16.0,22.0,2023-10-22 21:30:00
332,7,333,13.0,19.0,2023-10-22 22:00:00
333,7,334,10.0,19.0,2023-10-22 22:30:00
334,7,335,9.0,12.0,2023-10-22 23:00:00


# Post processing



In [ ]:
pd.DataFrame(all_optimised_shifts)

,shift,number_of_users
0,"(1, 1, 4, 16, 36)",5.0
1,"(1, 1, 4, 22, 42)",1.0
2,"(1, 6, 9, 20, 40)",1.0
3,"(1, 5, 8, 17, 37)",2.0
4,"(0, 4, 8, 18, 35)",1.0
...,...,...
77,"(0, 1, 5, 15, 31)",6.0
78,"(0, 7, 11, 3, 20)",1.0
79,"(1, 5, 8, 34, 54)",2.0
80,"(0, 7, 11, 6, 23)",1.0


In [ ]:
time = timedelta(hours = 5*0.5)

In [ ]:
str((pd.to_datetime(data['Date'].min()) + timedelta(days = 1, hours = 49*.5)).time())

'00:30:00'

In [ ]:
pd.to_datetime(data['Date'].min()).weekday()

0

In [ ]:
10%7

3

In [ ]:
time

datetime.timedelta(seconds=9000)

In [ ]:
optimised_shift = {}
all_optimised_shifts = []
week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

for shift in valid_time_shifts:
  if pulp.value(x[shift])>0.0:
    print(shift,pulp.value(x[shift]))

    worker_type = 'Full Time' if shift[0] == 0 else 'Part Time'

    start_day = (shift[1]-1) if shift[1] <=7 else (shift[1]%7 -1)
    end_day = (shift[2]-1) if shift[2] <=7 else (shift[2]%7 -1)
    if start_day < end_day:
      days = week[start_day:end_day+1]
    else:
      days = week[start_day:] + week[:end_day+1]

    start_time = str((pd.to_datetime(data['Date'].min()) + timedelta(hours = shift[3]*.5)).time())
    end_time = str((pd.to_datetime(data['Date'].min()) + timedelta(days = 1, hours = shift[4]*.5 + 0.5)).time())

    number_of_hours = shift[4]*.5 - shift[3]*.5 +0.5

    optimised_shift = {'worker_type': worker_type,'number_of_days': len(days),'number_of_hours': number_of_hours,'Days': days,'Start_time':start_time,'End_time':end_time ,'number_of_users': pulp.value(x[shift])}
    all_optimised_shifts.append(optimised_shift)

(1, 1, 4, 16, 36) 5.0
(1, 1, 4, 22, 42) 1.0
(1, 6, 9, 20, 40) 1.0
(1, 5, 8, 17, 37) 2.0
(0, 4, 8, 18, 35) 1.0
(0, 6, 10, 13, 29) 1.0
(0, 7, 11, 35, 52) 1.0
(1, 5, 8, 19, 39) 4.0
(0, 6, 10, 15, 31) 1.0
(0, 1, 5, 27, 43) 3.0
(1, 1, 4, 18, 38) 4.0
(1, 1, 4, 19, 39) 4.0
(1, 1, 5, 14, 26) 4.0
(0, 1, 5, 23, 40) 2.0
(0, 4, 8, 7, 24) 1.0
(1, 6, 9, 16, 36) 2.0
(0, 6, 10, 17, 34) 1.0
(0, 1, 5, 29, 45) 6.0
(1, 1, 4, 20, 40) 2.0
(0, 5, 9, 14, 30) 3.0
(1, 6, 9, 18, 38) 2.0
(0, 1, 5, 32, 48) 2.0
(1, 5, 8, 15, 35) 1.0
(0, 5, 9, 16, 33) 3.0
(0, 6, 10, 22, 39) 1.0
(0, 7, 11, 13, 29) 4.0
(0, 1, 5, 21, 38) 2.0
(0, 5, 9, 10, 26) 1.0
(1, 7, 10, 20, 40) 1.0
(0, 1, 5, 36, 53) 1.0
(1, 6, 9, 14, 34) 1.0
(1, 2, 6, 14, 26) 3.0
(1, 6, 9, 36, 56) 1.0
(0, 1, 5, 34, 50) 1.0
(1, 5, 9, 13, 25) 1.0
(0, 6, 10, 18, 35) 1.0
(1, 1, 4, 32, 52) 1.0
(0, 6, 10, 24, 41) 2.0
(1, 5, 8, 20, 40) 3.0
(0, 6, 10, 32, 48) 1.0
(0, 7, 11, 31, 47) 3.0
(0, 1, 5, 5, 22) 2.0
(1, 1, 5, 30, 42) 3.0
(0, 1, 5, 12, 28) 3.0
(0, 5, 9, 23, 40) 1.0
(

In [ ]:
start_day

0

In [ ]:
end_day

4

In [ ]:
output = pd.DataFrame(all_optimised_shifts)

In [ ]:
output.to_csv("schedule_dec1.csv",index = False)

In [ ]:
output

,worker_type,number_of_days,number_of_hours,Days,Start_time,End_time,number_of_users
0,Part Time,4,10.5,"[Monday, Tuesday, Wednesday, Thursday]",08:00:00,18:30:00,5.0
1,Part Time,4,10.5,"[Monday, Tuesday, Wednesday, Thursday]",11:00:00,21:30:00,1.0
2,Part Time,4,10.5,"[Saturday, Sunday, Monday, Tuesday]",10:00:00,20:30:00,1.0
3,Part Time,4,10.5,"[Friday, Saturday, Sunday, Monday]",08:30:00,19:00:00,2.0
4,Full Time,5,9.0,"[Thursday, Friday, Saturday, Sunday, Monday]",09:00:00,18:00:00,1.0
...,...,...,...,...,...,...,...
77,Full Time,5,8.5,"[Monday, Tuesday, Wednesday, Thursday, Friday]",07:30:00,16:00:00,6.0
78,Full Time,5,9.0,"[Sunday, Monday, Tuesday, Wednesday, Thursday]",01:30:00,10:30:00,1.0
79,Part Time,4,10.5,"[Friday, Saturday, Sunday, Monday]",17:00:00,03:30:00,2.0
80,Full Time,5,9.0,"[Sunday, Monday, Tuesday, Wednesday, Thursday]",03:00:00,12:00:00,1.0
